In [1]:
%matplotlib notebook
import os, sys
import logging
import random
import h5py
import shutil
import time
import argparse
import numpy as np
import sigpy.plot as pl
import torch
import sigpy as sp
import torchvision
from torch import optim
from tensorboardX import SummaryWriter
from torch.nn import functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib
# import custom libraries
from utils import transforms as T
from utils import subsample as ss
from utils import complex_utils as cplx
from utils.resnet2p1d import generate_model
from utils.flare_utils import roll
# import custom classes
from utils.datasets import SliceData
from subsample_fastmri import MaskFunc
from MoDL_single import UnrolledModel
import argparse
from models.SAmodel import MyNetwork
from models.Unrolled import Unrolled
from models.UnrolledRef import UnrolledRef
from models.UnrolledTransformer import UnrolledTrans
import matplotlib.pyplot as plt
from ImageFusionBlock import ImageFusionBlock
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from models.UnrolledFusion import UnrolledFusion
from fastmri.data import transforms, subsample
%load_ext autoreload
%autoreload 0

In [2]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [3]:
class DataTransform:
    """
    Data Transformer for training unrolled reconstruction models.
    """

    def __init__(self, mask_func, args, use_seed=False):
        self.mask_func = mask_func
        self.use_seed = use_seed
        self.rng = np.random.RandomState()
    
    def get_mask_func(self, factor):
        center_fractions = 0.08 * 4/factor # EquiSpacedMaskFuncRandomMaskFunc
        mask_func = subsample.RandomMaskFunc(
        center_fractions=[center_fractions],
        accelerations=[factor], 
        )
        return mask_func
    
    def __call__(self, kspace, target, reference, reference_kspace,slice):
        im_lowres = abs(sp.ifft(sp.resize(sp.resize(kspace,(256,24)),(256,160))))
        magnitude_vals = im_lowres.reshape(-1)
        k = int(round(0.05 * magnitude_vals.shape[0]))
        scale = magnitude_vals[magnitude_vals.argsort()[::-1][k]]
        kspace = kspace/scale
        target = target/scale
        # Convert everything from numpy arrays to tensors
        kspace_torch = cplx.to_tensor(kspace).float()   
        target_torch = cplx.to_tensor(target).float()  
        target_torch = T.ifft2(T.kspace_cut(T.fft2(target_torch),0.67,0.67)) 

        kspace_torch = T.awgn_torch(kspace_torch,10,L=1)
        ## Masking
        mask_func = self.get_mask_func(3)
        kspace_torch = T.kspace_cut(kspace_torch,0.67,0.67)
        kspace_torch = transforms.apply_mask(kspace_torch, mask_func)[0]
        # kspace_torch = kspace_torch*mask_torch # For poisson
        
        mask = np.abs(cplx.to_numpy(kspace_torch))!=0
        mask_torch = torch.stack([torch.tensor(mask).float(),torch.tensor(mask).float()],dim=2)
        
        ### Reference addition ###
        im_lowres_ref = abs(sp.ifft(sp.resize(sp.resize(reference_kspace,(256,24)),(256,160))))
        magnitude_vals_ref = im_lowres_ref.reshape(-1)
        k_ref = int(round(0.05 * magnitude_vals_ref.shape[0]))
        scale_ref = magnitude_vals_ref[magnitude_vals_ref.argsort()[::-1][k_ref]]
        reference = reference / scale_ref
        reference_torch = cplx.to_tensor(reference).float()
        reference_torch_kspace = T.fft2(reference_torch)
        reference_torch_kspace = T.kspace_cut(reference_torch_kspace,0.67,0.67)
        reference_torch = T.ifft2(reference_torch_kspace)
        

        return kspace_torch,target_torch,mask_torch, reference_torch 

In [4]:
def create_datasets(args):
    # Generate k-t undersampling masks
    train_mask = MaskFunc([0.08],[4])
    train_data = SliceData(
        root=str(args.data_path),
        transform=DataTransform(train_mask, args),
        sample_rate=1
    )
    return train_data
def create_data_loaders(args):
    train_data = create_datasets(args)
#     print(train_data[0])

    train_loader = DataLoader(
        dataset=train_data,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
    )
    return train_loader
def build_optim(args, params):
    optimizer = torch.optim.Adam(params, lr=args.lr, weight_decay=args.weight_decay)
    return optimizer

In [5]:
#Hyper parameters
params = Namespace()
params.data_path = "./registered_data/"
params.batch_size = 8
params.num_grad_steps = 4 #4
params.num_cg_steps = 8 #8
params.share_weights = True
params.modl_lamda = 0.05
params.lr = 0.00001
params.weight_decay = 0
params.lr_step_size = 10
params.lr_gamma = 0.3
params.epoch = 61
params.reference_mode = 0
params.reference_lambda = 0.1

In [6]:
train_loader = create_data_loaders(params)

/home/tal/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 5, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:

single_MoDL = UnrolledModel(params).to(device)
optimizer = build_optim(params, single_MoDL.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, params.lr_step_size, params.lr_gamma)
criterion = nn.MSELoss()


shared weights


In [8]:

import warnings

# Suppress specific deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message=".*Function sporco.util.tikhonov_filter is deprecated.*")

# Suppress specific user warnings from torchvision.models
warnings.filterwarnings("ignore", message=".*Using 'weights' as positional parameter.*")
warnings.filterwarnings("ignore", message=".*Arguments other than a weight enum or `None` for 'weights' are deprecated.*")
warnings.filterwarnings("ignore", category=DeprecationWarning, 
                        message=".*Function sporco.util.tikhonov_filter is deprecated; please use function sporco.signal.tikhonov_filter instead.*")

epochs_plot = []
losses_plot = []

for epoch in range(params.epoch):
    single_MoDL.train()
    avg_loss = 0.
    running_loss = 0.0
    for iter, data in enumerate(train_loader):
        input,target,mask,reference = data
        input = input.to(device)
        target = target.to(device)
        mask = mask.to(device)
        reference = reference.to(device)

        im_out = single_MoDL(input.float(),reference_image=reference,mask=mask)
        loss = criterion(im_out,target)
        running_loss = running_loss + loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss = 0.99 * avg_loss + 0.01 * loss.item() if iter > 0 else loss.item()
        if iter % 125 == 0:
            logging.info(
                f'Epoch = [{epoch:3d}/{params.epoch:3d}] '
                f'Iter = [{iter:4d}/{len(train_loader):4d}] '
                f'Loss = {loss.item():.4g} Avg Loss = {avg_loss:.4g}'
            )
    #Saving the model
    exp_dir = "L2_checkpoints_poisson_x2_MoDL_horizontal_LR_grant/"
    if epoch % 10 == 0:
        torch.save(
            {
                'epoch': epoch,
                'params': params,
                'model': single_MoDL.state_dict(),
                'optimizer': optimizer.state_dict(),
                'exp_dir': exp_dir
            },
            f=os.path.join(exp_dir, 'model_%d.pt'%(epoch))
    )
    running_loss = running_loss / len(train_loader)
    #scheduler.step(running_loss)
    scheduler.step()
    # Append epoch and average loss to plot lists
    epochs_plot.append(epoch)
    losses_plot.append(running_loss)

# Plotting the loss curve
plt.figure()
plt.plot(epochs_plot, losses_plot, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('SA unrolled with Reference L2 train Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(exp_dir, 'loss_plot_plato_down.png'))  # Save plot as an image

# Save all_losses to a file for later comparison
losses_file = os.path.join(exp_dir, 'all_losses.txt')
with open(losses_file, 'w') as f:
    for loss in losses_plot:
        f.write(f'{loss}\n')

INFO:root:Epoch = [  0/ 61] Iter = [   0/  67] Loss = 0.01383 Avg Loss = 0.01383
INFO:root:Epoch = [  1/ 61] Iter = [   0/  67] Loss = 0.01241 Avg Loss = 0.01241
INFO:root:Epoch = [  2/ 61] Iter = [   0/  67] Loss = 0.01139 Avg Loss = 0.01139
INFO:root:Epoch = [  3/ 61] Iter = [   0/  67] Loss = 0.01103 Avg Loss = 0.01103
INFO:root:Epoch = [  4/ 61] Iter = [   0/  67] Loss = 0.0103 Avg Loss = 0.0103
INFO:root:Epoch = [  5/ 61] Iter = [   0/  67] Loss = 0.01015 Avg Loss = 0.01015
INFO:root:Epoch = [  6/ 61] Iter = [   0/  67] Loss = 0.009444 Avg Loss = 0.009444
INFO:root:Epoch = [  7/ 61] Iter = [   0/  67] Loss = 0.008828 Avg Loss = 0.008828
INFO:root:Epoch = [  8/ 61] Iter = [   0/  67] Loss = 0.009559 Avg Loss = 0.009559
INFO:root:Epoch = [  9/ 61] Iter = [   0/  67] Loss = 0.009308 Avg Loss = 0.009308
INFO:root:Epoch = [ 10/ 61] Iter = [   0/  67] Loss = 0.009014 Avg Loss = 0.009014
INFO:root:Epoch = [ 11/ 61] Iter = [   0/  67] Loss = 0.009135 Avg Loss = 0.009135
INFO:root:Epoch = 

KeyboardInterrupt: 